In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-27"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11186,2024-04-27,Espanha Acb,15:45,Basquet Girona,Bilbao,1.48,2.68,160.5,1.85,1.93,-4.5,1.86,3.05,86PQRYxH,0.675676,0.373134,0.540541,0.518135,0.048810,0.4,0.8,NaN,NaN,293.912,267.346705,0.909615,183.184,6.994457,0.038183,160.052,347.310,148.57,186.40,0.0,0.0,0.0,0.0,0.407946,0.029930,0.342752,-3.40,-0.680,-0.705882,0.000000,0.0,0.000000,-2.67,-0.534,-3.146067,0.343032,0.2,-0.143032
11187,2024-04-27,Eua Nba,14:00,Orlando Magic,Cleveland Cavaliers,1.75,2.16,201.5,1.93,1.93,-3.5,2.00,2.65,Y9fdUhnS,0.571429,0.462963,0.518135,0.518135,0.034392,0.6,0.6,NaN,NaN,183.040,26.402315,0.144243,213.582,60.471100,0.283128,188.398,216.190,215.38,175.13,0.0,0.0,0.0,0.0,0.148293,0.000000,0.197686,-1.34,-0.268,-2.798507,0.605067,0.6,-0.005067,-5.00,-1.000,-1.160000,0.587867,0.4,-0.187867
11188,2024-04-27,Eua Nba,16:30,New Orleans Pelicans,Oklahoma City Thunder,1.97,1.89,208.5,1.93,1.93,-1.5,2.05,2.03,hnPMNbrm,0.507614,0.529101,0.518135,0.518135,0.036715,0.2,0.6,NaN,NaN,231.744,29.126502,0.125684,225.506,40.576908,0.179937,221.446,223.042,212.10,285.60,0.0,0.0,0.0,0.0,0.029310,0.000000,0.006932,-0.90,-0.180,-5.388889,0.584905,0.5,-0.084905,-1.19,-0.238,-3.739496,0.576345,0.4,-0.176345
11189,2024-04-27,Eua Nba,21:30,Los Angeles Lakers,Denver Nuggets,2.41,1.61,217.5,1.93,1.93,2.5,2.00,1.42,UVCmWrhd,0.414938,0.621118,0.518135,0.518135,0.036056,0.8,0.4,NaN,NaN,232.096,69.185584,0.298090,175.834,24.359382,0.138536,228.472,185.980,202.65,216.16,0.0,0.0,0.0,0.0,0.281436,0.000000,0.239837,0.62,0.124,11.370968,0.549504,0.6,0.050496,1.80,0.360,1.694444,0.563428,0.5,-0.063428
11190,2024-04-27,Alemanha Bbl,13:30,Ludwigsburg,Chemnitz,2.19,1.69,170.5,1.85,1.93,1.5,2.00,1.52,zkP9jYID,0.456621,0.591716,0.540541,0.518135,0.048337,0.2,0.8,NaN,NaN,135.888,32.820069,0.241523,163.382,94.883489,0.580746,117.106,204.060,97.80,332.00,0.0,0.0,0.0,0.0,0.182244,0.029930,0.192847,-1.09,-0.218,-5.458716,0.754897,0.7,-0.054897,-0.44,-0.088,-7.840909,0.000000,0.0,0.000000
11191,2024-04-27,Alemanha Pro A,14:30,Vechta 2,Munster,2.05,1.70,171.5,1.86,1.84,1.5,1.91,1.56,WlvuLGe1,0.487805,0.588235,0.537634,0.543478,0.076040,0.6,0.4,NaN,NaN,216.302,77.593741,0.358729,199.158,109.534108,0.549986,226.818,218.362,156.75,391.98,0.0,0.0,0.0,0.0,0.131993,0.007644,0.142644,2.87,0.574,1.829268,0.000000,0.0,0.000000,1.00,0.200,3.500000,0.000000,0.0,0.000000
11192,2024-04-27,Austrália Nbl1 Norte,06:30,Townsville Heat,Cairns Marlins,1.94,1.78,183.5,1.87,1.83,1.5,1.82,1.66,tvYQ8WZm,0.515464,0.561798,0.534759,0.546448,0.077262,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.060826,0.015289,0.065021,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11193,2024-04-27,Austrália Nbl1 Sul,06:30,Kilsyth,Nunawading,2.10,1.68,173.5,1.87,1.83,1.5,1.93,1.51,dv8Lcxi6,0.476190,0.595238,0.534759,0.546448,0.071429,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,212.76,148.63,0.0,0.0,0.0,0.0,0.157135,0.015289,0.172666,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11194,2024-04-27,Austrália Nbl1 Sul,07:00,Dandenong,Geelong,2.02,1.72,169.5,1.85,1.85,1.5,1.88,1.60,h61UeGLI,0.495050,0.581395,0.540541,0.540541,0.076445,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,154.23,0.0,0.0,0.0,0.0,0.113440,0.000000,0.113787,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11195,2024-04-27,Austrália Nbl1 Sul,07:00,Hobart Chargers,Ball

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,15:45,Espanha Acb,Basquet Girona,Bilbao,160.5,1.85,1.0000,Over
1,14:00,Eua Nba,Orlando Magic,Cleveland Cavaliers,201.5,1.93,1.0000,Over
2,16:30,Eua Nba,New Orleans Pelicans,Oklahoma City Thunder,208.5,1.93,1.0000,Over
3,04:30,Austrália Nbl1 Sul,Kilsyth F,Nunawading F,146.5,1.83,0.7298,Over
4,13:30,Espanha Leb Ouro,Fuenlabrada,Gipuzkoa,161.5,1.87,1.0000,Over
5,04:00,Filipinas Copa Das Filipinas,Blackwater Bossing,NorthPort,198.5,1.81,1.0000,Over
6,11:30,França Copa Da França,Dijon,Strasbourg,154.5,1.87,1.0000,Over
7,14:15,Grécia Liga De Basquete,AS Karditsas,AEK Athens,171.5,1.87,1.0000,Over
8,09:30,Irã Superliga,Zob Ahan,Tabiat,153.5,1.85,1.0000,Over
9,01:35,Japão B.League,Shimane,Hiroshima D.,156.5,1.83,0.9999,Over
